In [1]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np

In [6]:
#Step 1 read in dataframe from assignment 2 and 3
df = pd.read_csv('data_traffic_road_segments.csv', index_col=False)
df = df[df['Road name'] == 'N1']
df = df.drop(df.columns[[0]], axis=1)
df = df.rename(columns = {'Start LRP':'LRPName'})
df.drop_duplicates(subset = 'LRPName', keep = 'last', inplace = True)

listLRP = df['LRPName'].unique()
trafficmodes = list(df)[12:-1]

for mode in trafficmodes:
    df[mode + ' Difference'] = df[mode] - df[mode].shift(1)
df.iloc[0, 28:43] = df.iloc[0, 12:27].tolist()
trafficdifference = list(df)[-15:-1]
absolutevalues = list()
for traffic in trafficdifference:
    absolutevalues.append(abs(df[traffic].iloc[0]) )
df.iloc[[0],-15:-1] = absolutevalues
for difference, mode in zip(trafficdifference,trafficmodes):
    df[difference] = np.where(df[difference] < 0, -(1 - (df[mode] / df[mode].shift(1))), df[difference])
    df[mode + ' Create'] = np.where(df[difference] > 0, df[difference], 0) 
    df[mode + ' Destroy'] = np.where(df[difference] < 0, abs(df[difference]), 0) 
CreateDestroy = list(df)[-28:]
df2 = pd.read_csv('data_clean_N1.csv')
df2 = df2.drop(df2.columns[[0,1]], axis=1)
for column in CreateDestroy:
    df2[column] = df2.LRPName.map(df.set_index('LRPName')[column].to_dict())
df2['ControlTraffic'] = np.where(df2['LRPName'].isin(listLRP), 'ControlTraffic', '')
columns = list(df2)[0:-1]
columns.insert(2, list(df2)[-1:][0])
df2 = df2.reindex(columns, axis = 1)
roundcols = list(df2)[10:]
scale = 1
for col in roundcols:
    df2[col] = np.where(df2['ControlTraffic'] == 'ControlTraffic', round((df2[col]/scale),5),np.NaN)
df2['Add On Process'] = np.where(df2['Add On Process'] == 'BridgesDelay', 'BridgesDelay', '')
df2.Cat = df2.Cat.replace('0', '')
df2=df2.rename(columns = {'Add On Process':'BridgesDelay'})
df2['Index'] = df2.index + 1
for row in range(0, len(df2)):
    if df2.ControlTraffic.iloc[row] == 'ControlTraffic':
        df2.BridgesDelay.iloc[row] = 'DestroyTraffic'

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
df2[df2['ControlTraffic'] == 'ControlTraffic'].iloc[0:9]

,LRPName,Lat,ControlTraffic,Lon,Chainage,Description,Length,Cat,Next Node,BridgesDelay,Heavy Truck Create,Heavy Truck Destroy,Medium Truck Create,Medium Truck Destroy,Small Truck Create,Small Truck Destroy,Large Bus Create,Large Bus Destroy,Medium Bus Create,Medium Bus Destroy,Micro Bus Create,Micro Bus Destroy,Utility Create,Utility Destroy,Car Create,Car Destroy,Auto Rickshaw Create,Auto Rickshaw Destroy,Motor Cycle Create,Motor Cycle Destroy,Bi-Cycle Create,Bi-Cycle Destroy,Cycle Rickshaw Create,Cycle Rickshaw Destroy,Cart Create,Cart Destroy,Motorized Total Create,Motorized Total Destroy,Index
0,LRPS,23.706028,ControlTraffic,90.443333,0.000,0,0.0,,LRPSa,DestroyTraffic,28.10267,0.00000,503.61365,0.00000,251.46306,0.00000,430.90765,0.00000,241.66580,0.00000,233.24359,0.00000,57.15070,0.00000,303.11354,0.00000,502.49641,0.00000,198.09376,0.00000,62.56497,0.00000,224.13385,0.00000,0.0,0.0,2749.85083,0.00000,1
18,LRP009,23.705028,ControlTraffic,90.519333,8.503,0,0.0,,LRP009a,DestroyTraffic,9.07026,0.00000,563.14732,0.00000,510.07058,0.00000,271.17203,0.00000,349.34979,0.00000,282.63037,0.00000,10.37446,0.00000,115.67881,0.00000,206.17663,0.00000,63.02622,0.00000,20.19313,0.00000,72.34021,0.00000,0.0,0.0,2380.69647,0.00000,19
27,LRP012,23.691805,ControlTraffic,90.542611,11.497,0,0.0,,LRP012a,DestroyTraffic,2.81217,0.00000,80.70161,0.00000,57.61084,0.00000,53.11308,0.00000,44.71096,0.00000,39.02642,0.00000,5.10835,0.00000,31.68209,0.00000,53.61188,0.00000,19.75401,0.00000,6.26074,0.00000,22.42858,0.00000,0.0,0.0,388.13140,0.00000,28
32,LRP013,23.686527,ControlTraffic,90.550417,12.524,0,0.0,,LRP013a,DestroyTraffic,30.20397,0.00000,60.98483,0.00000,0.00000,0.69659,0.00000,0.46935,0.00000,0.74011,0.00000,0.30103,20.99100,0.00000,29.71731,0.00000,0.00000,0.61816,0.00000,0.82311,0.00000,0.91049,0.00000,0.87985,0.0,0.0,0.00000,0.38472,33
61,LRP022,23.625222,ControlTraffic,90.604972,21.629,0,0.0,,LRP022a,DestroyTraffic,4.68592,0.00000,80.67760,0.00000,16.59238,0.00000,26.75433,0.00000,11.03029,0.00000,25.89381,0.00000,6.25050,0.00000,32.05826,0.00000,19.43209,0.00000,3.31691,0.00000,0.53196,0.00000,2.55809,0.00000,0.0,0.0,226.69208,0.00000,62
89,LRP031,23.559972,ControlTraffic,90.654806,30.774,0,0.0,,LRP031a,DestroyTraffic,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,90
96,LRP033,23.548750,ControlTraffic,90.667833,32.607,0,0.0,,LRP033b,DestroyTraffic,0.00000,0.06473,0.00000,0.25370,0.00000,0.33537,0.00000,0.24375,0.00000,0.32674,0.00000,0.66628,0.00000,0.21157,0.00000,0.57944,0.00000,0.49406,0.00000,0.38327,0.00000,0.18430,0.00000,0.38692,0.0,0.0,0.00000,0.37268,97
128,LRP043,23.529416,ControlTraffic,90.751750,42.269,0,0.0,,LRP043a,DestroyTraffic,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.0,0.0,0.00000,0.15867,129
179,LRP067,23.497277,ControlTraffic,90.967472,65.726,0,0.0,,LRP067a,DestroyTraffic,0.00000,0.08911,95.41667,0.00000,0.00000,0.18887,35.25000,0.00000,0.00000,0.22538,50.16667,0.00000,1.08333,0.00000,6.83333,0.00000,193.08333,0.00000,64.91667,0.00000,12.83333,0.00000,8.75000,0.00000,0.0,0.0,391.00000,0.00000,180


In [4]:
df2.head()

,LRPName,Lat,ControlTraffic,Lon,Chainage,Description,Length,Cat,Next Node,BridgesDelay,Heavy Truck Create,Heavy Truck Destroy,Medium Truck Create,Medium Truck Destroy,Small Truck Create,Small Truck Destroy,Large Bus Create,Large Bus Destroy,Medium Bus Create,Medium Bus Destroy,Micro Bus Create,Micro Bus Destroy,Utility Create,Utility Destroy,Car Create,Car Destroy,Auto Rickshaw Create,Auto Rickshaw Destroy,Motor Cycle Create,Motor Cycle Destroy,Bi-Cycle Create,Bi-Cycle Destroy,Cycle Rickshaw Create,Cycle Rickshaw Destroy,Cart Create,Cart Destroy,Motorized Total Create,Motorized Total Destroy,Index
0,LRPS,23.706028,ControlTraffic,90.443333,0.000,0,0.0,,LRPSa,DestroyTraffic,28.10267,0.0,503.61365,0.0,251.46306,0.0,430.90765,0.0,241.6658,0.0,233.24359,0.0,57.1507,0.0,303.11354,0.0,502.49641,0.0,198.09376,0.0,62.56497,0.0,224.13385,0.0,0.0,0.0,2749.85083,0.0,1
1,LRPSa,23.702917,,90.450417,0.814,0,0.0,,LRPSb,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,LRPSb,23.702778,,90.450472,0.822,0,0.0,,LRP001,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,LRP001,23.702139,,90.451972,1.000,0,0.0,,LRP001a,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,LRP001a,23.698739,,90.458861,1.800,.,11.3,A,LRP002,BridgesDelay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [8]:
#Step 2 - Run this line to link first part the script with the second part
df2.to_csv('AutoCreateSimioAssignment4.csv')
#df2.to_excel('AutoCreateSimioAssignment4.xlsx')

,Unnamed: 0,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc,average_delay,Road_segment_no,Road_segment_label
0,0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.800,19.50,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.698739,90.458861,interpolate,NaN,N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right)
1,1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.40,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.694664,90.487775,interpolate,NaN,N1-3,Signboard - Shimrail (Left)R110 (Left)
2,2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.705060,90.523214,interpolate,NaN,N1-5,Katchpur - Madanpur (Left)N105 (Left)
3,3,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.880,12.20,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694391,90.537574,interpolate,NaN,N1-5,Katchpur - Madanpur (Left)N105 (Left)
4,4,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.20,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694302,90.537707,interpolate,NaN,N1-5,Katchpur - Madanpur (Left)N105 (Left)
5,5,N1,11.296,Box Culvert,LRP011a,NAYABARI KASPUR BOX CULVERT,8.30,A,101110,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,11.296,21.45,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.692360,90.540918,interpolate,NaN,N1-5,Katchpur - Madanpur (Left)N105 (Left)
6,6,N1,12.239,Box Culvert,LRP012a,KHAS PARA BOX CULVERT,9.30,A,101117,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,12.239,21.00,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.688412,90.548559,interpolate,NaN,N1-6,Madanpur - Langalband (Left)Z1061 (Left)
7,7,N1,12.253,Box Culvert,LRP012b,DAWAN BAG BOX CULVERT,6.10,A,101119,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,12.253,20.60,1987.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.688320,90.548650,interpolate,NaN,N1-6,Madanpur - Langalband (Left)Z1061 (Left)
8,8,N1,12.660,PC Girder Bridge,LRP013a,Madanpur Bridge.(L),27.50,A,119897,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,12.660,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Vitikandi,23.685583,90.551208,interpolate,NaN,N1-6,Madanpur - Langalband (Left)Z1061 (Left)
9,9,N1,12.660,PC Girder Bridge,LRP013a,MADAN PUR (R),26.30,A,109841,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,12.660,9.20,2003.0,1.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.685583,90.551208,interpolate,NaN,N1-6,Madanpur - Langalband (Left)Z1061 (Left)


In [9]:
#Step 3 -OLD STEPS OF LOADING IN DATA --> For assignment 3
df_traffic = pd.read_csv('Traffic_segment_data.csv', encoding='utf-8',sep = ",")
df_BMSS_traffic_link = pd.read_csv('Bridge_data_with_link_to_traffic.csv',encoding='utf-8',sep = ",")
df_simio = pd.read_csv('AutoCreateSimioAssignment4.csv',encoding='utf-8',sep = ",")

#calculate segment score (we weigth vulnerability and criticality equally)
df_traffic["segment_score"] = ""
df_traffic["Road_cat"] = ""
df_traffic["segment_score"] = df_traffic["segment_vulnerability_normalized"] * df_traffic["Economic_Traffic_normalized"]

#Selecting the five most important segments on the N1 (on criticality and vulnerability and get list of LRP of bridges to sample from)
df_traffic_assignment_4 = df_traffic[df_traffic["Road_Name"]=="N1"]
df_traffic_assignment_4 = df_traffic_assignment_4.sort_values(by='segment_score', ascending=False)
df_traffic_assignment_4 = df_traffic_assignment_4[0:5]
df_traffic_assignment_4 = df_traffic_assignment_4[['Road_Name','Link_No','Link_Name','Start LRP','End LRP','Total_A','Total_B','Total_C','Total_D' ,'segment_score']]

Top_five_segments = df_traffic_assignment_4["Link_No"]
Top_five_segments = Top_five_segments.values.tolist()

#obtain LRP rows of bridges within the top 5 five segments to sample from in Simio
df_most_important_bridges = df_BMSS_traffic_link[df_BMSS_traffic_link["Road_segment_no"].isin(Top_five_segments)]
df_most_important_bridges = df_most_important_bridges[['name','LRPName','Road_segment_no','condition' ]]

#Obtain list of all LRP's of the bridges within those top 5 segments
important_bridges_list = df_most_important_bridges["LRPName"]
important_bridges_list = important_bridges_list.values.tolist()
#df_most_important_bridges.to_csv('data_five_most_important_segments', sep=',')

df_traffic_assignment_4

,Road_Name,Link_No,Link_Name,Start LRP,End LRP,Total_A,Total_B,Total_C,Total_D,segment_score
35,N1,N1-36,Barabkunda Z1087-Fouzderhat Z1016 (Left),LRP209,LRP225,18,6,2,2,0.096062
6,N1,N1-7,Langalband - Mograpara Chowrasta (Left)Z1089 (...,LRP013,LRP013,4,9,2,0,0.051326
68,N1,N1-69,Whykong Z1133 - Teknaf,LRP433,LRP467,74,25,38,3,0.045194
11,N1,N1-12,Daudkandi Z1062 - Pennai Z1053 (Left),LRP033,LRP043,0,1,5,0,0.042074
23,N1,N1-24,Miabazar Z1046 - Chauddagram(Int.with Z1045) (...,LRP105,LRP105,7,12,0,0,0.041373


In [10]:
#Step 4 Iterate over the Simio dataframe to mark all important bridges for the add on process to break them down
df_simio["important_bridge"] = ""

for i in range(len(df_simio)):
    LRP = df_simio.iloc[i]['LRPName']
    if LRP in important_bridges_list and 
        df_simio.important_bridge.loc[i,] = "True"
    else:
        df_simio.important_bridge.loc[i,] = "False"

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [13]:
df_simio["important_bridge"].value_counts()

False    1400
True      154
Name: important_bridge, dtype: int64

In [21]:
#Step 5 Save resulting dataframe to use it as input for the Auto Creation of the model
df_simio.to_csv('AutoCreateSimioAssignment4.csv', sep=',')